In [6]:
import boto3
session = boto3.Session(profile_name="a1")
proj = "oslynstudio"
migration = "Feb2025" # change this

In [ ]:
from typing import List, Any

## Migration, coppy all the data fro old table to backup table
dynamodb = session.client('dynamodb', region_name="us-east-1")

lt: List[Any] = [
    {
        "tableName": f"SongTableBackup{migration}", 
        "key": { "name": "songId", "type": "HASH", "attr": "S"  }
    },
    {
        "tableName": f"BandTableBackup{migration}", 
        "key": { "name": "bandId", "type": "HASH", "attr": "S"  }
    },
    {
        "tableName": f"JamSessionTableBackup{migration}", 
        "key": { "name": "jamSessionId", "type": "HASH", "attr": "S"  }
    },
    {
        "tableName": f"SetListTableBackup{migration}", 
        "key": { "name": "setListId", "type": "HASH", "attr": "S"  }
    },
    {
        "tableName": f"UserTableBackup{migration}", 
        "key": { "name": "userId", "type": "HASH", "attr": "S"  }
    }
]

for t in lt:
    print("creating", t.get("tableName"), "...")
    dynamodb.create_table(
        TableName=t.get("tableName"),
        KeySchema=[{ 
            "AttributeName": t.get("key").get("name"),
            "KeyType": t.get("key").get("type")
        }],
        AttributeDefinitions=[{
            "AttributeName": t.get("key").get("name"),
            "AttributeType": t.get("key").get("attr")
        }],
        BillingMode="PAY_PER_REQUEST"
    )
print("complete!")

creating BandTableBackupFeb2025 ...
creating JamSessionTableBackupFeb2025 ...
creating SetListTableBackupFeb2025 ...
creating UserTableBackupFeb2025 ...
complete!


In [9]:
from boto3.dynamodb.types import TypeDeserializer
import time
unmarshall = TypeDeserializer()

## Migration, coppy all the data fro old table to backup table
dynamodb = session.client('dynamodb', region_name="us-east-1")
lts = [
    # {
    #     "oldTable": "oslynstudio-BandTable",
    #     "newTable": f"BandTableBackup{migration}"
    # },
    # {
    #     "oldTable": "oslynstudio-JamSessionTable",
    #     "newTable": f"JamSessionTableBackup{migration}"
    # },
    # {
    #     "oldTable": "oslynstudio-SetListTable",
    #     "newTable": f"SetListTableBackup{migration}"
    # },
    # {
    #     "oldTable": "oslynstudio-UserTable",
    #     "newTable": f"UserTableBackup{migration}"
    # },
    # {
    #     "oldTable": "oslynstudio-RecordingTable",
    #     "newTable": f"RecordingTableBackup{migration}"
    # },
    {
        "oldTable": "oslynstudio-SongTable",
        "newTable": f"SongTableBackup{migration}"
    },
]

for t in lts:
    print("Migrating", t.get("oldTable"), "-->", t.get("newTable"), "...")

    # Scan old table to get all items
    items = dynamodb.scan(
        TableName=t.get("oldTable"),
        Select="ALL_ATTRIBUTES"
    ).get("Items")

    print(items)
    time.sleep(5)

    # Migrate each item to the new table
    for item in items:        
        response = dynamodb.put_item(TableName=t.get("newTable"), Item=item)
        print(response)

    print(f"Migrated {len(items)} items successfully!")
    time.sleep(5)

Migrating oslynstudio-SongTable --> SongTableBackupFeb2025 ...
[{'album': {'S': 'Christ Our Hope In Life And Death'}, 'albumCover': {'S': 'https://i.scdn.co/image/ab67616d0000b273e81196f97485db7e65d8ec05'}, 'artist': {'S': 'Keith and Kristyn Getty'}, 'version': {'N': '1'}, 'songId': {'S': 'c26e2498-a84d-44fd-8824-99ae6cff4dc1'}, 'isApproved': {'BOOL': True}, 'userId': {'S': '9e7724c6-fbda-4a0c-874d-3841ae0848c1_usr'}, 'recordings': {'L': []}, 'chordSheet': {'S': '[Intro]  \nD   G  \n   \n[Verse 1]  \n            D       Dsus     D  \nWhat is our hope in life and death?  \nDsus    D            A  \nChrist alone, Christ alone.  \n        D        A/C# Bm  \nWhat is our only confidence?  \nG        D/A      A     D  \nThat our souls to Him belong.  \n              F#m             Bm  \nWho holds our days within His hand?  \n             Em7       D/F#  G  \nWhat comes, apart from His command?  \n              D/A     A7     Bm  \nAnd what will keep us to the end?  \n    G       D/A       

In [10]:
from boto3.dynamodb.types import TypeDeserializer
import time
unmarshall = TypeDeserializer()

## Migration, coppy all the data fro old table to backup table
dynamodb = session.client('dynamodb', region_name="us-east-1")
lts = [
    {
        "oldTable": "oslynstudio-BandTable",
        "newTable": f"BandTableBackup{migration}"
    },
    {
        "oldTable": "oslynstudio-JamSessionTable",
        "newTable": f"JamSessionTableBackup{migration}"
    },
    {
        "oldTable": "oslynstudio-RecordingTable",
        "newTable": f"RecordingTableBackup{migration}"
    },
    {
        "oldTable": "oslynstudio-SetListTable",
        "newTable": f"SetListTableBackup{migration}"
    },
    {
        "oldTable": "oslynstudio-SongTable",
        "newTable": f"SongTableBackup{migration}"
    },
    {
        "oldTable": "oslynstudio-UserTable",
        "newTable": f"UserTableBackup{migration}"
    },
]

for t in lts:
    print("Migrating back:", t.get("newTable"), "-->", t.get("oldTable"), "...")

    # Scan old table to get all items
    items = dynamodb.scan(
        TableName=t.get("newTable"),
        Select="ALL_ATTRIBUTES"
    ).get("Items")

    print(items)
    time.sleep(5)

    # Migrate each item to the new table
    for item in items:        
        response = dynamodb.put_item(TableName=t.get("oldTable"), Item=item)
        print(response)

    print(f"Migrated {len(items)} items successfully!")
    time.sleep(5)

print("Migration complete.")

Migrating back: BandTableBackupFeb2025 --> oslynstudio-BandTable ...
[{'policy': {'S': 'PUBLIC_JOIN'}, 'imageUrl': {'S': 'https://oslynstudio-s3stack-oslynstudiobucket4f3f730f-de5ih4nnn90k.s3.amazonaws.com/band/334bd95d-c532-4e60-b206-3e3fa29cbbe0_bnd/latest.jpg'}, 'userId': {'S': '9e7724c6-fbda-4a0c-874d-3841ae0848c1_usr'}, 'songIds': {'L': [{'S': 'ad387cad-4c88-483e-a70a-fae729dc346b'}, {'S': '09d9efaf-a939-472e-a0e9-6c09786d6fba'}, {'S': 'd8fda774-a75a-4f0c-83eb-0e71041b8d00'}, {'S': '76b3d511-41aa-4b65-93de-0af130792de1'}, {'S': '16d37897-a47c-49f4-8bcb-160f00b7ae19'}, {'S': 'db5d4069-9501-4853-a934-20d95c18a618'}, {'S': 'a9b5f993-5076-4fd9-adc1-534be6586843'}, {'S': '710a3d4a-b788-4ecd-b557-b7910b7f3623'}, {'S': '5618be0c-13c8-4ed3-8adf-bcba7ee88429'}, {'S': '7081a6be-c762-428a-9647-f8826504d590'}, {'S': '3679d250-6f68-4f89-a10e-67841014374a'}, {'S': 'de1f7f55-34b1-4e1a-8b52-394915e60b01'}, {'S': 'f5602113-b564-4768-9f98-e08cdb643798'}, {'S': '2547136d-0ff7-4ae8-b6d6-38c7b0b5d585'